[ref](https://github.com/scalanlp/breeze/wiki/Quickstart)

In [1]:
import breeze.linalg._

In [29]:
val x = DenseVector.zeros[Double](5)
x

DenseVector(0.0, 0.0, 0.0, 0.0, 0.0)

In [13]:
val y = SparseVector.zeros[Double](5)
y

SparseVector(5)()

In [30]:
x(1) = 2
x

DenseVector(0.0, 2.0, 0.0, 0.0, 0.0)

In [31]:
x(3 to 4) := .5
x

DenseVector(0.0, 2.0, 0.0, 0.5, 0.5)

In [32]:
Transpose(x)

Transpose(DenseVector(0.0, 2.0, 0.0, 0.5, 0.5))

In [33]:
x.t

Transpose(DenseVector(0.0, 2.0, 0.0, 0.5, 0.5))

In [7]:
val m = DenseMatrix.zeros[Int](5,5)
m

0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0

In [17]:
(m.rows, m.cols)

(5,5)

In [18]:
m(::,1)

DenseVector(0, 0, 0, 0, 0)

In [22]:
m(4,::):= DenseVector(1,2,3,4,5).t
m

0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
0  0  0  0  0
1  2  3  4  5

In [24]:
m(0 to 1, 0 to 1):= DenseMatrix((3,1),(-1,-2))
m

3   1   0  0  0
-1  -2  0  0  0
0   0   0  0  0
0   0   0  0  0
1   2   3  4  5

In [51]:
println( x + x )
println( x :+= 3.0)
println( x:* x )
println( x:*= 2.0)

DenseVector(6.0, 10.0, 6.0, 7.0, 7.0)
DenseVector(6.0, 8.0, 6.0, 6.5, 6.5)
DenseVector(36.0, 64.0, 36.0, 42.25, 42.25)
DenseVector(12.0, 16.0, 12.0, 13.0, 13.0)


In [50]:
var z = x + x
println(z :< x)

BitVector()


In [58]:
println(z dot x)
println(z.t * x)
println( sum(x) )
println( x.max )
println( argmax(x) )
//println( ceil(x) )
//println( floor(x) )

486.0
486.0
66.0
16.0
1


Name: Syntax Error.
Message: 
StackTrace: 

In [59]:
import breeze.stats.mean

In [61]:
val dm = DenseMatrix((1,2,3),(4,5,6))

In [63]:
val res = dm(::, *) + DenseVector(3,4)
res

4  5  6
8  9  10

In [82]:
breeze.stats.mean(dm(*, ::))

Name: Compile Error
Message: <console>:27: error: could not find implicit value for parameter impl: breeze.stats.mean.Impl[breeze.linalg.BroadcastedRows[breeze.linalg.DenseMatrix[Int],breeze.linalg.DenseVector[Int]],VR]
       breeze.stats.mean(dm(*, ::))
                        ^
StackTrace: 

In [65]:
import breeze.stats.distributions._

In [66]:
val poi = new Poisson(3.0)
val s = poi.sample(5)
s

Vector(4, 3, 1, 1, 2)

In [67]:
s map { poi.probabilityOf(_)}

Vector(0.16803135574154085, 0.22404180765538775, 0.14936120510359185, 0.14936120510359185, 0.22404180765538775)

In [71]:
val doublePoi = for(x <- poi) yield x.toDouble
breeze.stats.meanAndVariance(doublePoi.samples.take(1000))

MeanAndVariance(2.9650000000000003,3.0067817817817875,1000)

In [75]:
(poi.mean, poi.variance)

(3.0,3.0)

In [76]:
val expo = new Exponential(0.5)
expo.rate

0.5

In [78]:
expo.probability(0, Math.log(2) * expo.rate)

0.5

In [83]:
expo.probability(0, 1.5)

0.950212931632136

In [86]:
1 - Math.exp(-3.0)

0.950212931632136

In [89]:
val samples = expo.sample(2).sorted
expo.probability(samples(0), samples(1))

0.06364413613538611

In [90]:
breeze.stats.meanAndVariance(expo.samples.take(1000))

MeanAndVariance(2.032082561354724,4.058336673128916,1000)

In [91]:
(1 / expo.rate, 1/ (expo.rate * expo.rate))

(2.0,4.0)

In [97]:
import breeze.optimize._

In [99]:
val f = new DiffFunction[DenseVector[Double]] {
    def calculate(x: DenseVector[Double]) = {
        (norm((x - 3d) :^ 2d,1d),(x * 2d) - 6d);
    }
}

In [101]:
f.valueAt(DenseVector(3,3,3))

0.0

In [102]:
f.gradientAt(DenseVector(3,0,1))

DenseVector(0.0, -6.0, -4.0)

In [103]:
f.calculate(DenseVector(0,0))

(18.0,DenseVector(-6.0, -6.0))

In [105]:
def g(x: DenseVector[Double]) = (x - 3.0):^ 2.0 sum
g(DenseVector(0.,0.,0.))

Name: Compile Error
Message: <console>:1: error: identifier expected but ',' found.
g(DenseVector(0.,0.,0.))
                ^
<console>:1: error: identifier expected but ',' found.
g(DenseVector(0.,0.,0.))
                   ^
<console>:1: error: identifier expected but ')' found.
g(DenseVector(0.,0.,0.))
                      ^
StackTrace: 

In [107]:
val diffg = new ApproximateGradientFunction(g)
diffg.gradientAt(DenseVector(3,0,1))

DenseVector(1.000000082740371E-5, -5.999990000127297, -3.999990000025377)

In [108]:
val lbfgs = new LBFGS[DenseVector[Double]](maxIter=100, m=3) // m is the memory. anywhere between 3 and 7 is fine. The larger m, the more memory is needed.


In [109]:
val optimum = lbfgs.minimize(f,DenseVector(0,0,0))

In [110]:
f(optimum)

0.0

In [111]:
case class OptParams(batchSize:Int = 512,
                     regularization: Double = 1.0,
                     alpha: Double = 0.5,
                     maxIterations:Int = -1,
                     useL1: Boolean = false,
                     tolerance:Double = 1E-4,
                     useStochastic: Boolean= false) {
  // ...
}

[ref: vegas](https://github.com/vegas-viz/Vegas)

In [95]:
%AddDeps org.vegas-viz vegas_2.11 {vegas-version} --transitive

Marking org.vegas-viz:vegas_2.11:{vegas-version} for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps1849641686815801936/
-> https://repo1.maven.org/maven2
-> Failed to resolve org.vegas-viz:vegas_2.11:{vegas-version}
    -> not found: /tmp/toree_add_deps1849641686815801936/cache/org.vegas-viz/vegas_2.11/ivy-{vegas-version}.xml
    -> not found: https://repo1.maven.org/maven2/org/vegas-viz/vegas_2.11/%7Bvegas-version%7D/vegas_2.11-%7Bvegas-version%7D.pom


In [96]:
implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))

Name: Compile Error
Message: <console>:27: error: not found: value vegas
       implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))
                             ^
StackTrace: 

In [100]:
import vegas._

Name: Compile Error
Message: <console>:30: error: not found: value vegas
       import vegas._
              ^
StackTrace: 

In [113]:
import smile._

Name: Compile Error
Message: <console>:30: error: not found: value smile
       import smile._
              ^
StackTrace: 

In [114]:
import scalaz._